<a href="https://colab.research.google.com/github/spatank/Curiosity/blob/master/make_triangled_nets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.4.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools PyCall PyPlot"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.4.2 on the current Colab Runtime...
2021-01-17 20:55:07 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.4/julia-1.4.2-linux-x86_64.tar.gz [99093958/99093958] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
    Cloning default registries into `~/.julia`
    Cloning registry from "https://github.com/JuliaRegistries/General.git"
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed Artifacts ─────── v1.3.0
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed VersionParsing ── v1.2.0
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed SoftGlobalScope ─ v1.1.0
  Installed Parsers ───────── v1.0.15
  Installed IJulia ────────── v1.23.1
  Installed JLLWrappers ───── v1.2.0
  Installed Conda ─────────── v1.5.0
  Installed MbedTLS ───────── v1.0.3
  Installed JSON ──────────── v0.21.1
  Installed ZMQ ───────────── v1.2.1
######################################################################## 100.

## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [1]:
versioninfo()

Julia Version 1.4.2
Commit 44fa15b150* (2020-05-23 18:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Networks with Triangles

In [4]:
using Pkg
using Statistics
using LinearAlgebra
Pkg.add("Distances")
using Distances
Pkg.add("StatsBase")
using StatsBase
using Random
Pkg.add("Distributions")
using Distributions
Pkg.add("JLD")
using JLD
using Printf
Pkg.add("MAT")
using MAT
Pkg.add("JSON")
using JSON

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
  Installed Distances ─ v0.10.0
   Updating `~/.julia/environments/v1.4/Project.toml`
  [b4f34e82] + Distances v0.10.0
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [b4f34e82] + Distances v0.10.0
┌ Info: Precompiling Distances [b4f34e82-e78d-54a5-968a-f98e89d6e8f7]
└ @ Base loading.jl:1260
  Resolving package versions...
  Installed StatsBase ───────── v0.33.2
  Installed SortingAlgorithms ─ v0.3.1
  Installed Missings ────────── v0.4.4
  Installed DataAPI ─────────── v1.4.0
   Updating `~/.julia/environments/v1.4/Project.toml`
  [2913bbd2] + StatsBase v0.33.2
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [9a962f9c] + DataAPI v1.4.0
  [e1d29d7a] + Missings v0.4.4
  [a2af1166] + SortingAlgorithms v0.3.1
  [2913bbd2] + StatsBase v0.33.2
┌ Info: Precompiling StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91]
└ @ Base loading.jl:1260
  Resolving package versions...
  Installed QuadGK ─────────────────────── v2.4.1
  Installed OpenSpecFun_

In [5]:
function make_probtriangle_weighted(nNodes, p)
    ### p controls propensity of triangle formation
    ## triangle graph model
    nEdges = binomial(nNodes,2)
    adj = zeros(nNodes,nNodes)
    edges_left = copy(nEdges)
    while edges_left > 0       
        # Flip a coin
        r = rand(1)[1]
        if r < p  # See if there's a triangle and if there is, add it. If not, add edge randomly
            # Are any triangles possible?
            adj_2 = adj^2
            adj_2[diagind(adj_2)] .= 0
            possible_ot = Tuple.(findall(adj_2 .> 0))
            current_edges = Tuple.(findall(adj .> 0))
            # Remove n, n open triangles
            possible_ot = filter(x -> (x[1] != x[2]), possible_ot)
            # Check for non-closed open triangles
            open_triangles = []
            for pots in possible_ot
                if !(pots in current_edges)
                    open_triangles = [open_triangles; pots]
                end
            end
            if length(open_triangles) > 0
                # Then we can add an edge that completes a triangle, weighted by time of appearance.
                ot_weights = [adj_2[ot[1], ot[2]] for ot in open_triangles]
                new_edge = sample(open_triangles, Weights(ot_weights))
            else
                # Then we add a new edge randomly.
                # println("no open triangles")
                open_edges = Tuple.(findall(adj .== 0))
                open_edges = filter(x -> (x[1] != x[2]), open_edges)
                new_edge = sample(open_edges)
            end
        else  # add an edge randomly
            open_edges = Tuple.(findall(adj .== 0))
            open_edges = filter(x -> (x[1] != x[2]), open_edges)
            new_edge = sample(open_edges)
        end
        adj[new_edge[1], new_edge[2]] = edges_left
        adj[new_edge[2], new_edge[1]] = edges_left
        edges_left = edges_left-1
    end
    return adj
end

make_probtriangle_weighted (generic function with 1 method)

In [16]:
cd("/content/drive/MyDrive/Curiosity_IGT/")

In [17]:
pwd()

"/content/drive/MyDrive/Curiosity_IGT"

In [18]:
nNodes = 75
nReps = 5
all_ps = 0.1:0.1:0.9
for p in all_ps
  for iter in 1:nReps
    save_string = @sprintf("%.1f_%d.mat", p, iter)
    adj = make_probtriangle_weighted(nNodes, p)
    matwrite(save_string, Dict("p" => p, "adj" => adj))
  end
end

InterruptException: ignored